### 논문
- https://arxiv.org/pdf/2308.15928

### 입력

In [48]:
arr = [-3, 1, -2, 6, 4, 5 , 8, 7]

### reverse 정의

In [50]:
def reverse(arr, i, j, arr_inv):
    reversed = []
    for k in range(i, j + 1):
        reversed.append(-arr[k])
    for k in range(i, j + 1):
        value = reversed.pop()
        arr[k] = value
        arr_inv[abs(value)] = k

### 전처리

In [51]:
pi = [0] + arr + [len(arr) + 1]
n = pi[-1]
signed_pi_inv = [0] * (2*n)
pi_inv = [0] * (n + 1)
signed_pi = [0]
signed_pi_inv[0] = 0
pi_inv[0] = 0
pi_inv[n] = n
for i, num in enumerate(arr, 1):
    signed_pi_inv[-num] = len(signed_pi)
    signed_pi.append(-num)
    signed_pi_inv[num] = len(signed_pi)
    signed_pi.append(num)
    pi_inv[abs(num)] = i
signed_pi_inv[-n] = len(signed_pi)
signed_pi.append(-n)

edge_start_at = [None] * (2*n)
edge_end_at = [None] * (2*n)
black = [True] * n
for i in range(0, n):
    s = signed_pi_inv[i]
    e = signed_pi_inv[-(i + 1)]
    if s > e:
        s, e = e, s
    edge_start_at[s] = i
    edge_end_at[e] = i
    black[i] = ((1 if pi[i] >= 0 else -1) * pi[i + 1]) < 0

G = {i: {i} for i in range(n)}
S1 = []
S2 = []
for i in range(0, 2*n):
    if (v := edge_start_at[i]) != None:
        S1.append(v)
    elif (v := edge_end_at[i]) != None:
        while S1[-1] != v:
            S2.append(s := S1.pop())
            G[v].add(s)
            G[s].add(v)
        S1.pop()
        while S2:
            S1.append(S2.pop())

In [52]:
# non-singleton all-white connected component
from collections import deque

reversals = []
while True:
    nsawcc = []
    visited = [False] * n
    for i in range(n):
        if visited[i]:
            continue
        visited[i] = True
        connected = 0
        all_white = True
        queue = deque([i])
        while queue:
            here = queue.popleft()
            if black[here]:
                all_white = False
                break
            connected += 1
            for there in G[here]:
                if there == here or visited[there]:
                    continue
                visited[there] = True
                queue.append(there)
        if all_white and connected > 1:
            print(f'{i} is an element of non-singleton all-white connected component')
            if i == 0:
                nsawcc.append((pi_inv[i+1], pi_inv[i+1]))
            else:
                nsawcc.append((pi_inv[i], pi_inv[i]))
    if not nsawcc:
        break
    
    reversals.extend(nsawcc)
    for l, r in nsawcc:
        print(f'reversing [{l}, {r}]')
        reverse(pi, l, r, pi_inv)
        print(pi)
        print(pi_inv)

    signed_pi_inv = [0] * (2*n)
    pi_inv = [0] * (n + 1)
    signed_pi = [0]
    signed_pi_inv[0] = 0
    pi_inv[0] = 0
    pi_inv[n] = n
    for i, num in enumerate(arr, 1):
        signed_pi_inv[-num] = len(signed_pi)
        signed_pi.append(-num)
        signed_pi_inv[num] = len(signed_pi)
        signed_pi.append(num)
        pi_inv[abs(num)] = i
    signed_pi_inv[-n] = len(signed_pi)
    signed_pi.append(-n)

    edge_start_at = [None] * (2*n)
    edge_end_at = [None] * (2*n)
    black = [True] * n
    for i in range(0, n):
        s = signed_pi_inv[i]
        e = signed_pi_inv[-(i + 1)]
        if s > e:
            s, e = e, s
        edge_start_at[s] = i
        edge_end_at[e] = i
        black[i] = ((1 if pi[i] >= 0 else -1) * pi[i + 1]) < 0

    G = {i: {i} for i in range(n)}
    S1 = []
    S2 = []
    for i in range(0, 2*n):
        if (v := edge_start_at[i]) != None:
            S1.append(v)
        elif (v := edge_end_at[i]) != None:
            while S1[-1] != v:
                S2.append(s := S1.pop())
                G[v].add(s)
                G[s].add(v)
            S1.pop()
            while S2:
                S1.append(S2.pop())

5 is an element of non-singleton all-white connected component
reversing [6, 6]
[0, -3, 1, -2, 6, 4, -5, 8, 7, 9]
[0, 2, 3, 1, 5, 6, 4, 8, 7, 9]
7 is an element of non-singleton all-white connected component
reversing [8, 8]
[0, -3, 1, -2, 6, 4, -5, 8, -7, 9]
[0, 2, 3, 1, 5, 6, 4, 8, 7, 9]


In [53]:
print(pi)
print(pi_inv)
print(signed_pi)
print(signed_pi_inv)
G

[0, -3, 1, -2, 6, 4, -5, 8, -7, 9]
[0, 2, 3, 1, 5, 6, 4, 8, 7, 9]
[0, 3, -3, -1, 1, 2, -2, -6, 6, -4, 4, -5, 5, -8, 8, -7, 7, -9]
[0, 4, 5, 1, 10, 12, 8, 16, 14, 17, 13, 15, 7, 11, 9, 2, 6, 3]


{0: {0, 2, 3},
 1: {1, 2},
 2: {0, 1, 2},
 3: {0, 3, 5, 6},
 4: {4},
 5: {3, 5, 6},
 6: {3, 5, 6, 7, 8},
 7: {6, 7, 8},
 8: {6, 7, 8}}

### toggle 정의

In [54]:
def toggle(vertices, G, black):
    print('@toggle START')
    print(f'\t{G=}')
    print(f'\t{black=}')
    ret = set(vertices)
    for neighbor in vertices:
        G[neighbor] = G[neighbor] ^ vertices | {neighbor}
        black[neighbor] = not black[neighbor]
    print(f'>>>\n\t{G=}')
    print(f'\t{black=}')
    print('@toggle END')
    return ret

### reversal_sort 정의

In [55]:
def reversal_sort(G, black):
    S1, S2 = [], []
    V = set()
    
    for v in G:
        if len(G[v]) != 1 or black[v]:
            V.add(v)
    
    print(f'initial {V=}\n')
            
    def toggle_black():
        for v in V:
            if black[v]:
                break
        affected = toggle(G[v], G, black)
        for v in set(V):
            if len(G[v]) == 1 and not black[v]:
                V.remove(v)
        S1.append((v, affected))
        print(f'=== toggled, {V=}\n')

    def undo_last_toggle():
        w, to_undo = S1.pop()
        affected = toggle(to_undo, G, black)
        print(f'=== undone toggle, {V=}\n')
        return w, affected

    while any(black[v] for v in V):
        toggle_black()
    
    while V:
        while all(not black[v] for v in V):
            undone = undo_last_toggle()
            S2.append(undone)
        while any(black[v] for v in V):
            toggle_black()
        if not black[S2[-1][0]]:
            undo_last_toggle()
        
    print()
    print(f'{S1=}')
    print(f'{S2=}')
    print(f'{V=}')
    print(f'{G=}')
    print(f'{black=}')

    ret = []
    for sort in S1 + S2[::-1]:
        ret.append(sort[0])
    return ret

### 테스트

In [56]:
ov_reversals = reversal_sort(G, black)
print(ov_reversals)

initial V={0, 1, 2, 3, 5, 6, 7, 8}

@toggle START
	G={0: {0, 2, 3}, 1: {1, 2}, 2: {0, 1, 2}, 3: {0, 3, 5, 6}, 4: {4}, 5: {3, 5, 6}, 6: {3, 5, 6, 7, 8}, 7: {8, 6, 7}, 8: {8, 6, 7}}
	black=[True, True, True, True, False, True, True, True, True]
>>>
	G={0: {0}, 1: {1, 2}, 2: {1, 2, 3}, 3: {2, 3, 5, 6}, 4: {4}, 5: {3, 5, 6}, 6: {3, 5, 6, 7, 8}, 7: {8, 6, 7}, 8: {8, 6, 7}}
	black=[False, True, False, False, False, True, True, True, True]
@toggle END
=== toggled, V={1, 2, 3, 5, 6, 7, 8}

@toggle START
	G={0: {0}, 1: {1, 2}, 2: {1, 2, 3}, 3: {2, 3, 5, 6}, 4: {4}, 5: {3, 5, 6}, 6: {3, 5, 6, 7, 8}, 7: {8, 6, 7}, 8: {8, 6, 7}}
	black=[False, True, False, False, False, True, True, True, True]
>>>
	G={0: {0}, 1: {1}, 2: {2, 3}, 3: {2, 3, 5, 6}, 4: {4}, 5: {3, 5, 6}, 6: {3, 5, 6, 7, 8}, 7: {8, 6, 7}, 8: {8, 6, 7}}
	black=[False, False, True, False, False, True, True, True, True]
@toggle END
=== toggled, V={2, 3, 5, 6, 7, 8}

@toggle START
	G={0: {0}, 1: {1}, 2: {2, 3}, 3: {2, 3, 5, 6}, 4: {4}, 5: {

In [59]:
sorted_pi = list(pi)
sorted_pi_inv = list(pi_inv)
print(sorted_pi)
for v in ov_reversals:
    a = min(sorted_pi_inv[v], sorted_pi_inv[v + 1])
    b = max(sorted_pi_inv[v], sorted_pi_inv[v + 1])
    if pi[a] + pi[b] == 1:
        b -= 1
    else:
        a += 1
    reverse(sorted_pi, a, b, sorted_pi_inv)
    print(a, b, sorted_pi, sorted_pi_inv)

[0, -3, 1, -2, 6, 4, -5, 8, -7, 9]
8 9 [0, -3, 1, -2, 6, 4, -5, 8, -9, 7] [0, 2, 3, 1, 5, 6, 4, 9, 7, 8]
7 7 [0, -3, 1, -2, 6, 4, -5, -8, -9, 7] [0, 2, 3, 1, 5, 6, 4, 9, 7, 8]
7 7 [0, -3, 1, -2, 6, 4, -5, 8, -9, 7] [0, 2, 3, 1, 5, 6, 4, 9, 7, 8]
7 7 [0, -3, 1, -2, 6, 4, -5, -8, -9, 7] [0, 2, 3, 1, 5, 6, 4, 9, 7, 8]
8 9 [0, -3, 1, -2, 6, 4, -5, -8, -7, 9] [0, 2, 3, 1, 5, 6, 4, 8, 7, 9]
5 8 [0, -3, 1, -2, 6, 7, 8, 5, -4, 9] [0, 2, 3, 1, 8, 7, 4, 5, 6, 9]


: 